In [1]:
import sys
import os
import pickle
import pandas as pd
import json
import datetime
import time
import copy
import numpy as np

py_file_location = '../PrivatePackages'
sys.path.append(os.path.abspath(py_file_location))
from sklearn.model_selection import train_test_split

import JiaoCheng
import NingXiang

/Applications/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
LABEL = 'logret_normalised_by_day'
MODEL = 'lstmr'

In [3]:
curated_data = pd.read_csv('../data/curated_data.csv')
curated_data = curated_data[['date', 
                        'stock',
                        'log_ret',
                        'closePrice_lag_1', 
                        'log_ret_normalised_by_day_lag_1', 
                        'closePriceNorm_lag_1', 
                        'log_ret_lag_1', 
                        'mean_log_ret_lag_1',
                        'mean_closePrice_lag_1',
                        'mean_closePriceNorm_lag_1',
                        'mean_log_ret_normalised_by_day_lag_1', 
                        'pos_log_ret_lag_1',
                        'pos_closePrice_lag_1',
                        'pos_closePriceNorm_lag_1',
                        'pos_log_ret_normalised_by_day_lag_1']]
curated_data['date'] = pd.to_datetime(curated_data['date'])

In [4]:
date = [datetime.datetime(year=2022, month=12, day = 31) + datetime.timedelta(days=i) for i in range(250)]
val_date_start = date[175]
test_date_start = date[175+38]

In [5]:
curated_data['date'] = pd.to_datetime(curated_data['date'])
curated_data = curated_data.dropna()

train = curated_data[(curated_data['date'] < val_date_start)]
val = curated_data[(curated_data['date'] < test_date_start) & (curated_data['date'] >= val_date_start)]
test = curated_data[(curated_data['date'] >= test_date_start)]

In [6]:
def data_factory(data, lag):
    x_list, y_list = list(), list()

    for id, stock_data in data.groupby('stock'):
        stock_data.sort_values(by='date', inplace=True)
        stock_data.drop(['stock', 'date'], axis=1, inplace=True)
        stock_data.index = range(len(stock_data))

        for i in range(len(stock_data)):
            
            if i < LAG-1:
                continue

            x = stock_data.iloc[i-LAG+1:i+1].values.copy()
            x[-1:, 0] = 0 # mask last day's return
            y = stock_data.loc[i]['log_ret']
            
            x_list.append(x)
            y_list.append(y)
    
    return x_list, y_list

In [19]:
LAG = 5

train_x, train_y = data_factory(train, LAG)
val_x, val_y = data_factory(val, LAG)
test_x, test_y = data_factory(test, LAG)

In [8]:
# lstm = LSTM.LSTM_pt(lstm_hidden_layer_n_neurons = 50,
#                  lstm_n_hidden_layers = 1,
#                  bi_lstm = False,
#                  n_hidden_layers = 1,
#                  batch_size = 128,
#                  learning_rate = 0.01,
#                  dense_hidden_layer_n_neurons = 50,
#                  activation = 'relu',
#                  num_epochs = 100,
#                  random_state = 19260817,
#                  dropout_prob = 0.1,
#                  batch_normalisation = False,
#                  verbose = True,
#                  loss_function='MSE',
#                  dense_layer_type = 'Dense',
#                  grad_clip = False)

In [9]:
# lstm.fit(train_x, train_y)

In [10]:
# lstm.predict(val_x)

In [20]:
jiaocheng = JiaoCheng.JiaoCheng()

from models.LSTM import LSTMR_pt as LSTMR

jiaocheng.read_in_model(LSTMR, 'Regression')

jiaocheng.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

parameter_choices = {
    'lstm_hidden_layer_n_neurons': [25, 50, 100, 200],
    'lstm_n_hidden_layers': [1, 2, 3],
    'bi_lstm': [False, True],
    'n_hidden_layers': [1, 2, 3],
    'batch_size': [32, 64, 128, 256, 512],
    'learning_rate': [0.001, 0.01, 0.1, 1],
    'dense_hidden_layer_n_neurons': [25, 50, 100, 200],
    'activation': ['relu', 'sigmoid', 'tanh', 'softmax'],
    'num_epochs': [100, 200, 500, 1000],
    'dropout_prob': [0, 0.1, 0.2, 0.3, 0.4, 0.5],
    'batch_normalisation': [False, True],
    'dense_layer_type': ['Dense', 'Residual'],
    'grad_clip': [False, True]

}

jiaocheng.set_hyperparameters(parameter_choices)

jiaocheng.set_non_tuneable_hyperparameters({'verbose' : True, 'random_state' : 19260817, 'loss_function':'MSE',})

jiaocheng.set_tuning_order(['lstm_n_hidden_layers', 'n_hidden_layers', 'lstm_hidden_layer_n_neurons', 'dense_hidden_layer_n_neurons', 
                            'bi_lstm', 'activation', 'dense_layer_type',
                            'dropout_prob', 'batch_size', 'num_epochs', 'batch_normalisation', 'grad_clip', 'learning_rate'])

jiaocheng.set_hyperparameter_default_values({
    'lstm_hidden_layer_n_neurons': 50,
    'lstm_n_hidden_layers': 1,
    'bi_lstm': False,
    'n_hidden_layers': 1,
    'batch_size': 128,
    'learning_rate': 0.01,
    'dense_hidden_layer_n_neurons': 50,
    'activation': 'relu',
    'num_epochs': 100,
    'dropout_prob': 0.1,
    'batch_normalisation': False,
    'dense_layer_type': 'Dense',
    'grad_clip': False}
)

try:
    jiaocheng.read_in_tuning_result_df(f'../models/tuning/jiaocheng_{MODEL}_{LABEL}.csv')
except:
    pass

jiaocheng.set_tuning_result_saving_address(f'../models/tuning/jiaocheng_{MODEL}_{LABEL}.csv')
jiaocheng.set_best_model_saving_address(f'../models/tmp_models/jiaocheng_{MODEL}_{LABEL}.pickle')

jiaocheng.tune()

jiaocheng.view_best_combo_and_score()

print('\n\n')

JiaoCheng Initialised
Successfully read in model <module 'models.LSTM' from '/Users/tg.chenny/Desktop/3. Self Projects/2. PythonProject/Incomplete/DeepTrade/PrivatePackages/models/LSTM.py'>, which is a Regression model
Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data
Successfully recorded hyperparameter choices
Successfully recorded non_tuneable_hyperparameter choices
Successfully set tuning output address
Successfully set best model output address

Default combo: [1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 0, 0, 0] 


ROUND 1

Round 1 
Hyperparameter: lstm_n_hidden_layers (index: 1) 



TypeError: 'module' object is not callable